<a href="https://colab.research.google.com/github/kyleborot/lstm-and-sentiment-analysis/blob/main/LLaMa3_1_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install transformers datasets accelerate evaluate bitsandbytes huggingface_hub trl peft torch tensorboard

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

In [3]:
import os
import random
import functools
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import evaluate
from IPython.core.display import HTML
import requests
import time
import datetime
import nltk
import re
HTML("""<style>.container{width:100% !important;} </style>""")

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score, precision_recall_fscore_support

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from datasets import Dataset, DatasetDict
from torch.utils.data import Dataset as tds, DataLoader as tdl
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import RobertaTokenizer, RobertaForSequenceClassification, DataCollatorWithPadding, RobertaModel, RobertaForMaskedLM, DataCollatorForLanguageModeling
import torch.nn as nn

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
with open('/content/drive/MyDrive/News Classification Kaggle/News_Category_Dataset_v3.json') as f:
  df = pd.read_json(f, orient='records', lines=True)

In [5]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(text):
  encodings = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
  return {
      'input_ids': encodings['input_ids'].squeeze().tolist(),
      'attention_mask': encodings['attention_mask'].squeeze().tolist()
  }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [6]:
df['text'] = df['headline'] + ' ' + df['short_description']
tokens = df['text'].apply(lambda x: tokenize_function(x))
tokens_df = pd.DataFrame(tokens.tolist())
tokens_df.columns = ['tokens_input_ids', 'tokens_attention_mask']
df = df.join(tokens_df)
df = df.drop(columns=['text'])
df = df.rename(columns={'tokens_input_ids': 'input_ids', 'tokens_attention_mask': 'attention_mask'})


In [7]:
df = df.loc[:,~df.columns.duplicated()]

In [8]:
def to_list(data):
  if isinstance(data, torch.Tensor):
    return data.tolist()
  return data
df['input_ids'] = df['input_ids'].apply(to_list)
df['attention_mask'] = df['attention_mask'].apply(to_list)

In [9]:
def convert_to_dataset(df):
  return Dataset.from_pandas(df)

In [10]:
required_columns = ['input_ids', 'attention_mask']
for col in required_columns:
  if col not in df.columns:
    raise ValueError(f"Required column '{col}' not found in the DataFrame.")

In [11]:
dataset = convert_to_dataset(df)

In [12]:
train_df, eval_df = train_test_split(df, test_size=0.1, random_state=42)

In [13]:
train_dataset = convert_to_dataset(train_df)
eval_dataset = convert_to_dataset(eval_df)

In [20]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/News Classification Kaggle/results',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    logging_dir='/content/drive/MyDrive/News Classification Kaggle/logs',
    logging_steps=10,
    save_steps=5000,
    evaluation_strategy='epoch',
    save_total_limit=2,
    learning_rate=2e-5,
    gradient_accumulation_steps=4,
    fp16=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
model = RobertaForMaskedLM.from_pretrained("roberta-base")

In [22]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'precision': precision_recall_fscore_support(p.label_ids, preds, average='weighted')[0],
        'recall': precision_recall_fscore_support(p.label_ids, preds, average='weighted')[1],
        'f1': precision_recall_fscore_support(p.label_ids, preds, average='weighted')[2],
    }

In [23]:
from transformers import DataCollatorForLanguageModeling
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer)
)

In [ ]:
torch.cuda.empty_cache()
for epoch in range(10):
  trainer.train()
  trainer.save_model(f'/content/drive/MyDrive/News Classification Kaggle/results/epoch_{epoch}')
  torch.cuda.empty_cache()
  trainer.evaluate()
  torch.cuda.empty_cache()


Epoch,Training Loss,Validation Loss


In [19]:
torch.cuda.empty_cache()

In [ ]:
model.save_pretrained('/content/drive/MyDrive/News Classification Kaggle/roberta_model')
tokenizer.save_pretrained('/content/drive/MyDrive/News Classification Kaggle/roberta_tokenizer')

In [ ]:
import time
time.sleep(10)
from google.colab import runtime
runtime.unassign()

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)